In [1]:
from binascii import hexlify
import requests
import json

### Get cookie

In [70]:
def getCookie():
    url = "https://aes.cryptohack.org/flipping_cookie/get_cookie/"
    r = requests.get(url)
    ct = (json.loads(r.text))["cookie"]
    return ct


In [71]:
cipher = getCookie()
cipher

'855de393fee040a8c3d911ee28675bc1ffdef6335eb068721e015a2da9355d1e38cfb51796870daa0c789534ecfb663b'

In [72]:
iv = cipher[:32]
cookie = cipher[32:]
len(cookie), len(iv), iv, cookie 

(64,
 32,
 '855de393fee040a8c3d911ee28675bc1',
 'ffdef6335eb068721e015a2da9355d1e38cfb51796870daa0c789534ecfb663b')

### Get the first block of the cookie

In [78]:
cookie_encoded = "admin=False;expiry="[:16]
cookie_encoded, hexlify(cookie_encoded.encode())

('admin=False;expi', b'61646d696e3d46616c73653b65787069')

In [79]:
P = hexlify(cookie_encoded.encode())
P, iv

(b'61646d696e3d46616c73653b65787069', '855de393fee040a8c3d911ee28675bc1')

### Calc the value `T` just before XOR in the decript block

In [80]:
T = int(P, 16) ^ int(iv, 16)
T_hex = '{:x}'.format(T)
T_hex, len(T_hex)

('e4398efa90dd06c9afaa74d54d1f2ba8', 32)

### Get the first block we need 

In [81]:
cookie_target = "admin=True; expiry="[:16]
cookie_target, hexlify(cookie_target.encode())

('admin=True; expi', b'61646d696e3d547275653b2065787069')

### Calc `IV` for the block we need

In [82]:
P2 = hexlify(cookie_target.encode())
IV_new = int(P2, 16) ^ int(T_hex, 16)
IV_new_hex = '{:x}'.format(IV_new)
IV_new_hex, len(IV_new_hex)

('855de393fee052bbdacf4ff528675bc1', 32)

### Query for the FLAG

In [84]:
def checkAdmin(cookie, iv):
    url = f"https://aes.cryptohack.org/flipping_cookie/check_admin/{cookie}/{iv}/"
    r = requests.get(url)
    #ct = (json.loads(r.text))["flag"]
    return r.text # ct

checkAdmin(cookie, IV_new_hex)

'{"flag":"crypto{4u7h3n71c4710n_15_3553n714l}"}\n'